# Read data from Redis

In [25]:
import pandas as pd
import matplotlib.pyplot as plt
import redis
import json
import datetime as dt
import numpy as np
from typing import Tuple

Connect to Redis

In [4]:
store = redis.Redis()

## Read data

In [5]:
personId = 2
rawList = store.lrange(f'personData{personId}', 0, -1) # -1 is the last element

type(rawList)
# rawList

data = [json.loads(d.decode()) for d in rawList] # Decode each JSON
data

[{'birthdate': '1976',
  'disabled': True,
  'firstname': 'Elżbieta',
  'id': 12,
  'lastname': 'Kochalska',
  'trace': {'id': 13532805062019,
   'name': 'ela',
   'sensors': [{'anomaly': False, 'id': 0, 'value': 1023},
    {'anomaly': False, 'id': 1, 'value': 1023},
    {'anomaly': False, 'id': 2, 'value': 1023},
    {'anomaly': False, 'id': 3, 'value': 12},
    {'anomaly': False, 'id': 4, 'value': 11},
    {'anomaly': False, 'id': 5, 'value': 12}]},
  'timestamp': 1577299474.470599},
 {'birthdate': '1976',
  'disabled': True,
  'firstname': 'Elżbieta',
  'id': 12,
  'lastname': 'Kochalska',
  'trace': {'id': 13532605062019,
   'name': 'ela',
   'sensors': [{'anomaly': False, 'id': 0, 'value': 1023},
    {'anomaly': False, 'id': 1, 'value': 911},
    {'anomaly': False, 'id': 2, 'value': 1023},
    {'anomaly': False, 'id': 3, 'value': 1023},
    {'anomaly': False, 'id': 4, 'value': 378},
    {'anomaly': False, 'id': 5, 'value': 14}]},
  'timestamp': 1577299473.222482},
 {'birthdate': '

## Attemt to put data to df

In [6]:
current_id = 1

key = f'personData{current_id}'

rawList = store.lrange(key, 0, 5)
data = [json.loads(d.decode()) for d in rawList]

values = {'time': [] , 'sensor_0': [], 'sensor_1': [], 'sensor_2': [], 'sensor_3': [], 'sensor_4': [], 'sensor_5': []}

for value in data:
    values['time'].append(dt.datetime.fromtimestamp(value['timestamp']))
    sensors = value['trace']['sensors']
    
    for s in sensors:
        id = s['id']
        key = f'sensor_{id}'
        values[key].append(s['value'])
#     print(sensors)
    
    
df = pd.DataFrame(values)
df

,time,sensor_0,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5
0,2019-12-25 19:44:35.666830,1023,947,24,23,21,23
1,2019-12-25 19:44:34.397186,1023,1006,1023,1007,55,34
2,2019-12-25 19:44:33.135688,1023,872,36,1023,210,1023
3,2019-12-25 19:44:31.828870,1023,990,1023,29,29,30
4,2019-12-25 19:44:30.538582,26,26,25,1023,527,1023
5,2019-12-25 19:44:29.268148,1023,761,1023,27,27,28


### Unsuccessful attempt

In [29]:
data[0]['trace']['sensors']

[{'anomaly': False, 'id': 0, 'value': 370},
 {'anomaly': False, 'id': 1, 'value': 363},
 {'anomaly': False, 'id': 2, 'value': 14},
 {'anomaly': False, 'id': 3, 'value': 1023},
 {'anomaly': False, 'id': 4, 'value': 1023},
 {'anomaly': False, 'id': 5, 'value': 1023}]

In [36]:
tmp = [[(d['trace']['sensors'][i]['value'], d['trace']['sensors'][i]['anomaly']) for i in range(6)] for d in data]

tmp

[[(370, False),
  (363, False),
  (14, False),
  (1023, False),
  (1023, False),
  (1023, False)],
 [(335, False),
  (456, False),
  (15, False),
  (1023, False),
  (1023, False),
  (1023, False)],
 [(380, False),
  (497, False),
  (15, False),
  (1023, False),
  (1023, False),
  (1023, False)],
 [(133, False),
  (341, False),
  (14, False),
  (1023, False),
  (1023, False),
  (1023, False)],
 [(510, False),
  (418, False),
  (13, False),
  (1023, False),
  (1023, False),
  (910, False)],
 [(368, False),
  (256, False),
  (13, False),
  (1023, False),
  (1023, False),
  (1023, False)],
 [(113, False),
  (364, False),
  (14, False),
  (1023, False),
  (1023, False),
  (1023, False)],
 [(1023, False),
  (1023, False),
  (1023, False),
  (13, False),
  (12, False),
  (13, False)],
 [(1023, False),
  (1023, False),
  (16, False),
  (983, False),
  (393, False),
  (1023, False)],
 [(1023, False),
  (1023, False),
  (1023, False),
  (12, False),
  (13, False),
  (13, False)],
 [(1023, False)

In [45]:
fields = {f'Value {i}': [] for i in range(1, 6)}
fields.update({f'Anomaly {i}': [] for i in range(1, 6)})
fields

{'Value 1': [],
 'Value 2': [],
 'Value 3': [],
 'Value 4': [],
 'Value 5': [],
 'Anomaly 1': [],
 'Anomaly 2': [],
 'Anomaly 3': [],
 'Anomaly 4': [],
 'Anomaly 5': []}

In [69]:
flattened = [[]] * 12 
print(flattened)

for row in tmp:
    i = 0
    for sensor in row:
        flattened[i].append(sensor[0])
#         print(i)
#         print(sensor[0])
        i += 1

        flattened[i].append(sensor[1])
#         print(i)
#         print(sensor[1])
        i += 1
#         print(i)
        
flattened[1]

[[], [], [], [], [], [], [], [], [], [], [], []]


[370,
 False,
 363,
 False,
 14,
 False,
 1023,
 False,
 1023,
 False,
 1023,
 False,
 335,
 False,
 456,
 False,
 15,
 False,
 1023,
 False,
 1023,
 False,
 1023,
 False,
 380,
 False,
 497,
 False,
 15,
 False,
 1023,
 False,
 1023,
 False,
 1023,
 False,
 133,
 False,
 341,
 False,
 14,
 False,
 1023,
 False,
 1023,
 False,
 1023,
 False,
 510,
 False,
 418,
 False,
 13,
 False,
 1023,
 False,
 1023,
 False,
 910,
 False,
 368,
 False,
 256,
 False,
 13,
 False,
 1023,
 False,
 1023,
 False,
 1023,
 False,
 113,
 False,
 364,
 False,
 14,
 False,
 1023,
 False,
 1023,
 False,
 1023,
 False,
 1023,
 False,
 1023,
 False,
 1023,
 False,
 13,
 False,
 12,
 False,
 13,
 False,
 1023,
 False,
 1023,
 False,
 16,
 False,
 983,
 False,
 393,
 False,
 1023,
 False,
 1023,
 False,
 1023,
 False,
 1023,
 False,
 12,
 False,
 13,
 False,
 13,
 False,
 1023,
 False,
 725,
 False,
 1023,
 False,
 1023,
 False,
 1023,
 False,
 350,
 False,
 1023,
 False,
 1023,
 False,
 1023,
 False,
 207,
 False

## Colors for table cells

### Map value to color

In [32]:
def map_value_to_RGB(value : float) -> Tuple[float, float, float]:
    smallest, biggest = 0, 1023 # From doctor Zawadzki's documentation
    value_range = [smallest, biggest]

    red_range = [85, 85]
    green_range = [85, 180]
    blue_range = [85, 85]

    r = np.interp(value, value_range, red_range)
    g = np.interp(value, value_range, green_range)
    b = np.interp(value, value_range, blue_range)
    
    return r, g, b

map_value_to_RGB(760)

(85.0, 155.5767350928641, 85.0)